In [1]:
from evaluation.mol_structure import mol_to_smiles, nx_to_mol,list_of_smiles_to_nx_graphs
# Generate graphs for demonstration purposes
import utils.graph_generators as gen
import torch
import dgl
import networkx as nx
from rdkit.Chem import Descriptors
from evaluation.moses.metrics import get_all_metrics
from evaluation.evaluator import Evaluator
from evaluation.gin_evaluation import load_feature_extractor, MMDEvaluation
import copy
import fuckit
import os 
import sys
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)

from rdkit import Chem
from rdkit.Chem.rdchem import ChiralType, BondType, BondDir
import enum


In [2]:
#rom data.smiles.carcinogens import test_smiles
reference_smiles_list = []
with open('data/smiles/carcinogens/test_smiles.txt') as my_file:
    for line in my_file:
        reference_smiles_list.append(line)
        
        

In [3]:
reference_nx_graphs=list_of_smiles_to_nx_graphs(reference_smiles_list)


In [6]:
device=torch.device('cpu')
metrics_type=['nn','molecular','structural']
structural_statistic='nspdk'  #options ['WL' , 'nspdk', 'orbits, 
#'degree', 'clustering' ,'spectral']
#structural_statistic=['WL' , 'nspdk', 'orbits', 'degree', 'clustering' ,'spectral']



discrete_node_label_name='label' #leave it blank if this does not exist, default: 'label'
 #leave blank if this does not exist, default: 'label'
continuous_node_label_name=''  #leave it blank if this does not exist default: 'attr'
discrete_edge_label_name='label'  #leave it blank if this does not exist  ,default: 'label'
continuous_edge_label_name='' #leave it blank if this does not exist , default: 'attr'
#dicrete labels should be set to 'label'
#continous labels should be set to 'attr'


def preprocess(nx_dataset,discrete_node_label_name, continuous_node_label_name,discrete_edge_label_name,  continuous_edge_label_name, set_label_equal_to_attribute=False):
    processed_dataset=[]
    for G in nx_dataset:
        if (discrete_node_label_name!='label'):
            dict=nx.get_node_attributes(G, discrete_node_label_name)
            if dict!='':   
                nx.set_node_attributes(G, dict, 'label') 
            else:  pass
        if (discrete_edge_label_name!='label'):
            dict=nx.get_edge_attributes(G, discrete_edge_label_name)
            if dict!='':  
                nx.set_edge_attributes(G, dict, 'label')
            else:  pass
        if (continuous_node_label_name!='attr'):
            dict=nx.get_node_attributes(G, continuous_node_label_name)
            if dict!='':   
                nx.set_node_attributes(G, dict, 'attr') 
            else:  pass
        if (continuous_edge_label_name!='attr'):
            dict=nx.get_edge_attributes(G, continuous_edge_label_name)
            if dict!='':  
                nx.set_edge_attributes(G, dict, 'attr') 
            else:  pass
        if set_label_equal_to_attribute: 
            dict=nx.get_node_attributes(G, discrete_node_label_name)
            nx.set_node_attributes(G, dict, 'attr')
            dict=nx.get_edge_attributes(G, discrete_edge_label_name)
            nx.set_edge_attributes(G, dict, 'attr') 
        H = nx.MultiDiGraph(G)
        processed_dataset.append(H)
    return processed_dataset  

def get_datasets(reference_graphs,generated_graphs):
    generated_graphs=preprocess(reference_graphs,discrete_node_label_name, continuous_node_label_name, discrete_edge_label_name,continuous_edge_label_name,set_label_equal_to_attribute=True)
    reference_graphs=preprocess(reference_graphs,discrete_node_label_name, continuous_node_label_name, discrete_edge_label_name,continuous_edge_label_name,set_label_equal_to_attribute=True)
    return generated_graphs,reference_graphs

generated_graphs,reference_graphs= get_datasets(reference_nx_graphs[:10],reference_nx_graphs[10:])

reference_graphs_dgl = [ dgl.from_networkx(g,node_attrs=['label','attr'], edge_attrs=['label','attr']).to(device) for g in reference_graphs] # Convert graphs to DGL f,rom NetworkX
generated_graphs_dgl = [ dgl.from_networkx(g,node_attrs=['label','attr'], edge_attrs=['label','attr']).to(device) for g in generated_graphs] # Convert graphs to DGL from NetworkX
#reference_graphs_dgl = grids
#generated_graphs_dgl = lobsters
print(reference_graphs_dgl[0].ndata['attr'][0])
print(reference_graphs_dgl[0].edata['attr'][0])


metrics={}
if  'nn' in metrics_type:
   # assert continuous_node_label_name!='' ,'You need continuous features to be able to compute nn-based metrics'
    eval=Evaluator(feature_extractor ='gin',device='cpu', edge_feat_loc='attr' , node_feat_loc='attr', input_dim=1,edge_feat_dim=4, hidden_dim=36)
    nn_metrics=eval.evaluate_all(generated_dataset=reference_graphs_dgl,reference_dataset=reference_graphs_dgl)
    metrics.update(nn_metrics)
        
if 'structural' in metrics_type  :
    def fun():
        eval=Evaluator(feature_extractor ='mmd-structure',device='cpu' ,statistic=structural_statistic)
        structural_metrics=eval.evaluate_all(generated_dataset=generated_graphs_dgl,reference_dataset=reference_graphs_dgl)
        metrics.update(structural_metrics)
    #try:  
    fun()
   # except: print('Cannot compute these metrics for this type of graphs')
    
if 'molecular'  in metrics_type:
    #@fuckit
    def func(reference_smiles, generated_smieles):
        """
        reference_graphs_mol= [nx_to_mol(g,edge_label='label', node_label='label') for g in reference_graphs]
        generated_graphs_mol=[nx_to_mol(g,edge_label='label', node_label='label') for g in generated_graphs]
        reference_graphs_smiles=mol_to_smiles(reference_graphs_mol)
        generated_graphs_smiles=mol_to_smiles(generated_graphs_mol)
        """
        mol_metrics=get_all_metrics(gen=reference_smiles_list,train=reference_smiles_list)
        metrics.update(mol_metrics)
    try: 
        func(reference_smiles_list,reference_smiles_list)
    except: 
        print('Cannot compute molecular metrics for this type of graphs. You might neeed  to manually change the definition of nx_to_mol:)\n')
else: None
    
print(metrics)

tensor([6, 0, 0])
tensor([0, 1, 0, 0])
orthogonal
tensor([[ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [16.],
        [ 8.],
        [ 8.],
        [ 8.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 7.],
        [ 7.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 7.],
        [ 7.],
        [ 6.],
        [ 6.],
        [ 8.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [16.],
        [ 8.],
        [ 8.],
        [ 8.],
        [ 6.],
        [ 6.],
        [16.],
        [ 8.],
        [ 8.],
        [ 8.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 8.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 8.],
        [ 8.],
        [ 6.],
     

c:\Users\cs843\Documents\PhD\2023\Proposal\GGM-metrics\evaluation\moses\metrics\metrics.py:216: UserWarning: Can't compute unique@1000.gen contains only 56 molecules
  warnings.warn(
c:\Users\cs843\Documents\PhD\2023\Proposal\GGM-metrics\evaluation\moses\metrics\metrics.py:216: UserWarning: Can't compute unique@10000.gen contains only 56 molecules
  warnings.warn(


{'activations_time': 0.048561811447143555, 'fid': -4.6621475178199034e-06, 'fid_time': 0.05759620666503906, 'kid': -0.974175, 'kid_time': 11.595475435256958, 'precision': 1.0, 'recall': 1.0, 'f1_pr': 1.00001, 'precision_time': 0.07925772666931152, 'recall_time': 0.07925772666931152, 'f1_pr_time': 0.07925772666931152, 'density': 1.2000000000000002, 'coverage': 1.0, 'f1_dc': 1.090919173552968, 'density_time': 0.06556987762451172, 'coverage_time': 0.06556987762451172, 'f1_dc_time': 0.06556987762451172, 'mmd_rbf': 0, 'mmd_rbf_time': 0.07455658912658691, 'mmd_linear': 0.0, 'mmd_linear_time': 0.048561811447143555, 'nspdk_mmd': 0.0, 'nspdk_mmd_time': 0.19187426567077637, 'valid': 1.0, 'unique@1000': 1.0, 'unique@10000': 1.0, 'FCD/Test': 40.840599325778264, 'SNN/Test': 0.3579252077532666, 'Frag/Test': 0.6398663638024442, 'Scaf/Test': 0.02446344582055726, 'FCD/TestSF': 41.88350282196939, 'SNN/TestSF': 0.3388982641377619, 'Frag/TestSF': 0.039878026715357784, 'Scaf/TestSF': 0.0003477798127208276,

In [48]:
reference_graphs_dgl[0].ndata['label'][0].sum()


tensor(6)

In [30]:
 feat_dict = {node: str(reference_graphs_dgl[0].ndata['label'][node].item().sum(1).unsqueeze(1)) for node in range(reference_graphs_dgl[0].number_of_nodes())}

ValueError: only one element tensors can be converted to Python scalars

In [ ]:
m=torch.Tensor([1, 2])
m.unsqueeze(1)

tensor([[1.],
        [2.]])

In [ ]:
t=torch.Tensor([[1, 2, 3], [4, 5, 6]])
t.sum(1).unsqueeze(1)

tensor([[ 6.],
        [15.]])

In [4]:
a=gen.load_zinc(n_samples=10)
grids=a
lobsters=a
#print(len(grids[0].ndata['attr']))
#print(grids[0].ndata['attr']  , grids[0].ndata['feat'])

In [5]:
lobsters_nx = [ dgl.to_networkx(g, node_attrs=['attr','feat'], edge_attrs=['attr','feat'])  for g in a]
grids_nx = [dgl.to_networkx(g, node_attrs=['attr','feat'], edge_attrs=['attr','feat']) for g in a]

#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device=torch.device('cpu')

grids = [ dgl.from_networkx(g,node_attrs=['attr','feat'], edge_attrs=['attr','feat']).to(device) for g in grids_nx] # Convert graphs to DGL from NetworkX
lobsters = [ dgl.from_networkx(g,node_attrs=['attr','feat'], edge_attrs=['attr','feat']).to(device) for g in lobsters_nx] # Convert graphs to DGL from NetworkX

In [34]:
len(lobsters[0].ndata['feat'][0].size())==0


True

In [12]:
(lobsters[0].ndata['attr'][0].size())

torch.Size([28])

In [50]:
lobsters[0].edata['feat'][0].sum()

tensor(1)

In [5]:
reference_graphs_dgl[0].ndata['label']

tensor([[ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [16,  0,  0],
        [ 8,  0,  0],
        [ 8,  0,  0],
        [ 8,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 7,  0,  0],
        [ 7,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 7,  0,  0],
        [ 7,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 8,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [16,  0,  0],
        [ 8,  0,  0],
        [ 8,  0,  0],
        [ 8,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [16,  0,  0],
        [ 8,  0,  0],
        [ 8,  0,  0],
        [ 8,  0,  0],
        [ 6,  0,  0],
        [ 6,  0,  0],
        [ 

In [31]:
srcs, dests, eids = reference_graphs_dgl[0].edges('all')
feat_dict={}
for src, dest, eid in zip(srcs, dests, eids):
    print(src.item(), dest.item(),eid)
    feat_dict[(src.item(), dest.item())] = str(reference_graphs_dgl[0].edata['label'][eid].nonzero().item())

0 1 tensor(0)
1 0 tensor(1)
1 2 tensor(2)
1 53 tensor(3)
2 1 tensor(4)
2 3 tensor(5)
3 2 tensor(6)
3 4 tensor(7)
4 3 tensor(8)
4 5 tensor(9)
4 52 tensor(10)
5 4 tensor(11)
5 6 tensor(12)
5 7 tensor(13)
5 8 tensor(14)
6 5 tensor(15)
7 5 tensor(16)
8 5 tensor(17)
8 9 tensor(18)
9 8 tensor(19)
9 10 tensor(20)
9 51 tensor(21)
10 9 tensor(22)
10 11 tensor(23)
11 10 tensor(24)
11 12 tensor(25)
12 11 tensor(26)
12 13 tensor(27)
12 50 tensor(28)
13 12 tensor(29)
13 14 tensor(30)
14 13 tensor(31)
14 15 tensor(32)
15 14 tensor(33)
15 16 tensor(34)
15 48 tensor(35)
16 15 tensor(36)
16 17 tensor(37)
17 16 tensor(38)
17 18 tensor(39)
18 17 tensor(40)
18 19 tensor(41)
18 47 tensor(42)
19 18 tensor(43)
19 20 tensor(44)
19 46 tensor(45)
20 19 tensor(46)
20 21 tensor(47)
21 20 tensor(48)
21 22 tensor(49)
22 21 tensor(50)
22 23 tensor(51)
22 44 tensor(52)
23 22 tensor(53)
23 24 tensor(54)
24 23 tensor(55)
24 25 tensor(56)
25 24 tensor(57)
25 26 tensor(58)
25 43 tensor(59)
26 25 tensor(60)
26 27 tensor(6

In [14]:
metrics

{'activations_time': 0.012001276016235352,
 'fid': -5.29678833061098e-06,
 'fid_time': 0.015000581741333008,
 'kid': -1.1708288,
 'kid_time': 0.02800607681274414,
 'precision': 1.0,
 'recall': 1.0,
 'f1_pr': 1.00001,
 'precision_time': 0.05299806594848633,
 'recall_time': 0.05299806594848633,
 'f1_pr_time': 0.05299806594848633,
 'density': 1.2000000000000002,
 'coverage': 1.0,
 'f1_dc': 1.090919173552968,
 'density_time': 0.030998945236206055,
 'coverage_time': 0.030998945236206055,
 'f1_dc_time': 0.030998945236206055,
 'mmd_rbf': 0,
 'mmd_rbf_time': 0.041004180908203125,
 'mmd_linear': 0.0,
 'mmd_linear_time': 0.012001276016235352,
 'nspdk_mmd': 0.0,
 'nspdk_mmd_time': 0.19000530242919922,
 'wl_mmd': 0.0,
 'wl_mmd_time': 0.022936344146728516,
 'degree_mmd': 0,
 'degree_mmd_time': 56.915164947509766,
 'clustering_mmd': 0,
 'clustering_mmd_time': 56.11217641830444,
 'orbits_mmd': 0,
 'orbits_mmd_time': 0.06999516487121582,
 'spectral_mmd': 0,
 'spectral_mmd_time': 33.338955879211426,
 '

In [ ]:
reference_graphs[0].nodes(data=True)